In [45]:
import pandas as pd
df = pd.read_csv("../data/raw/sofifa_players_all_years.csv",
                low_memory=False)
df.drop_duplicates(inplace=True)

pd.set_option('display.max_rows', 70)
pd.set_option('display.max_columns', 70)

In [46]:
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(" / ", "_").str.replace(" & ", "_").str.replace(" ", "_")

df['height(cm)'] = df["height"].str.split("cm").str[0].astype(int)
df['weight(kg)'] = df["weight"].str.split("kg").str[0].astype(int)

foot_mapping = {"Left": 1, "Right": 2}
df['foot'] = df['foot'].map(foot_mapping)

df[['start_part', 'end_year']] = df['team_contract'].str.split(' ~ ', expand=True)
df[['team', 'start_year']] = df['start_part'].str.extract(r'([A-Za-z]+)(\d{4})')

In [47]:
import re

def name_position(row):
    pattern = r'[A-Z]{2,}'
    match = re.search(pattern, row)
    if match:
        location = match.start()
        name = row[:location].strip()
        
        return name
    else:
        return None

df['name'] = df['name'].apply(name_position)
df["name"] = df['name'].str.lower()

In [48]:
df = df[df['best_position'] != "GK"].copy()

In [49]:
def money(row):
    row = row.split('€')[1]
    if "M" in row:
        value = float(row.replace("M", "")) * 1_000_000
    elif "K" in row:
        value = float(row.replace("K", "")) * 1_000
    else:
        value = float(row)

    return value

df['value(€)'] = df['value'].apply(money)
df['wage(€)'] = df['wage'].apply(money)
df['release_clause(€)'] = df['release_clause'].apply(money)

In [50]:
deleted_col = ["unnamed:_0", "unnamed:_76", "id", "joined", 
                "loan_date_end", "club_kit_number", "club_position", 
                "traits", "traits.1", "acceleration_type", "playstyles", 
                "playstyles_+", "attacking_work_rate", "defensive_work_rate",
                "real_face", "team_contract", "end_year", "height", "weight", 
                "body_type", "season_code", "start_year", "start_part",
                "best_position", "value", "wage", "release_clause"]
df.drop(columns=deleted_col, inplace=True)

In [51]:
col_order = ['name', 'age', 'team', 'height(cm)', 'weight(kg)', 'season'] + [col for col in df.columns if col not in ['name', 'age', 'team', 'height(cm)', 'weight(kg)', 'season']]
df = df[col_order]

In [52]:
import shutil
# shutil.copyfile("../../model_1/data/raw/total_raw/total_fielders.csv", "../data/raw/total_fielders.csv")

In [53]:
df_fbref = pd.read_csv("../data/raw/total_fielders.csv")
def convert_season(season):
    season_str = str(season).zfill(4)  
    first_two = int(season_str[:2])
    if first_two >= 90:
        return 1900 + first_two
    else:
        return 2000 + first_two
    

df_fbref['season'] = df_fbref['season'].apply(convert_season)
df_fbref = df_fbref[df_fbref['season'] >= 2007].copy()
df_fbref.reset_index(drop=True, inplace=True)
df_fbref.sort_values(by=['season', "Performance_Gls"], ascending=[False, False], inplace=True)

In [54]:
def change_name(row):
    row = row.lower()
    names = row.split(" ")
    if len(names) >= 2:
        first_name = names[0]
        last_name = names[-1]
        new_name = f"{first_name[0]}. {last_name}"
        return new_name
    else:
        return row

In [63]:
df_fbref['player'] = df_fbref['player'].apply(change_name)

In [64]:
common_names = set(df.name).intersection(set(df_fbref.player))

In [73]:
df_fbref[df_fbref['player'].str.contains("junior", case=False, na=False)].head()

,league,season,team,player,nation,pos,age,born,Playing Time_MP,Playing Time_Starts,Playing Time_Min,Playing Time_90s,Performance_Gls,Performance_Ast,Performance_G+A,Performance_G-PK,Performance_PK,Performance_PKatt,Performance_CrdY,Performance_CrdR,Per 90 Minutes_Gls,Per 90 Minutes_Ast,Per 90 Minutes_G+A,Per 90 Minutes_G-PK,Per 90 Minutes_G+A-PK,90s,Standard_Gls,Standard_Sh,Standard_SoT,Standard_SoT%,Standard_Sh/90,Standard_SoT/90,Standard_G/Sh,Standard_G/SoT,Standard_Dist,...,Performance_PKwon,Performance_PKcon,Performance_OG,Expected_xG,Expected_npxG,Expected_xAG,Expected_npxG+xAG,Progression_PrgC,Progression_PrgP,Progression_PrgR,Per 90 Minutes_xG,Per 90 Minutes_xAG,Per 90 Minutes_xG+xAG,Per 90 Minutes_npxG,Per 90 Minutes_npxG+xAG,Standard_FK,Expected_npxG/Sh,Expected_G-xG,Expected_np:G-xG,xAG,Expected_xA,Expected_A-xAG,PrgP,Carries_PrgC,Receiving_Rec,Receiving_PrgR,Team Success (xG)_onxG,Team Success (xG)_onxGA,Team Success (xG)_xG+/-,Team Success (xG)_xG+/-90,Team Success (xG)_On-Off,Performance_Recov,Aerial Duels_Won,Aerial Duels_Lost,Aerial Duels_Won%
42932,ITA-Serie A,2024,Bologna,s. iling-junior,ENG,"FW,MF",20.0,2003.0,7,0,118,1.3,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.76,0.00,0.76,0.76,0.76,1.3,1.0,4.0,1.0,25.0,3.05,0.76,0.25,1.00,19.0,...,0.0,0.0,0.0,0.1,0.1,0.1,0.3,5.0,7.0,27.0,0.11,0.09,0.19,0.11,0.19,0.0,0.03,0.9,0.9,0.1,0.2,-0.1,7.0,5.0,79.0,27.0,1.4,1.7,-0.2,-0.19,-0.54,8.0,0.0,3.0,0.0
40656,ITA-Serie A,2023,Juventus,s. iling-junior,ENG,"DF,MF",19.0,2003.0,24,4,810,9.0,1.0,2.0,3.0,1.0,0.0,0.0,1.0,0.0,0.11,0.22,0.33,0.11,0.33,9.0,1.0,9.0,4.0,44.4,1.00,0.44,0.11,0.25,15.5,...,0.0,0.0,0.0,0.7,0.7,1.3,2.0,29.0,25.0,64.0,0.08,0.14,0.23,0.08,0.23,0.0,0.08,0.3,0.3,1.3,1.6,0.7,25.0,29.0,343.0,64.0,12.8,8.1,4.7,0.52,-0.17,40.0,9.0,3.0,75.0
38104,ITA-Serie A,2022,Juventus,s. iling-junior,ENG,"DF,FW",18.0,2003.0,12,1,318,3.5,1.0,1.0,2.0,1.0,0.0,0.0,1.0,0.0,0.28,0.28,0.57,0.28,0.57,3.5,1.0,4.0,1.0,25.0,1.13,0.28,0.25,1.00,22.4,...,0.0,0.0,0.0,0.7,0.7,0.2,1.0,11.0,18.0,36.0,0.21,0.07,0.28,0.21,0.28,0.0,0.19,0.3,0.3,0.2,0.4,0.8,18.0,11.0,114.0,36.0,6.8,2.8,4.0,1.15,0.86,18.0,1.0,3.0,25.0
36708,ESP-La Liga,2022,Mallorca,l. junior,CIV,"FW,MF",31.0,1990.0,5,0,52,0.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.6,0.0,2.0,1.0,50.0,3.46,1.73,0.00,0.00,17.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,5.0,0.03,0.00,0.03,0.03,0.03,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,2.0,20.0,5.0,1.0,1.3,-0.3,-0.51,-0.22,7.0,4.0,1.0,80.0
29305,ESP-La Liga,2019,Mallorca,l. junior,CIV,"MF,FW",28.0,1990.0,35,24,2188,24.3,4.0,0.0,4.0,1.0,3.0,3.0,3.0,0.0,0.16,0.00,0.16,0.04,0.04,24.3,4.0,38.0,9.0,23.7,1.56,0.37,0.03,0.11,17.1,...,0.0,3.0,0.0,6.2,3.8,1.3,5.1,103.0,48.0,193.0,0.25,0.05,0.31,0.16,0.21,0.0,0.10,-2.2,-2.8,1.3,1.6,-1.3,48.0,103.0,751.0,193.0,22.8,38.1,-15.3,-0.63,-0.12,119.0,48.0,50.0,49.0


In [83]:
df.rename(columns={"name": "player"}, inplace=True)

In [82]:
print(len(df[df['name'].isin(common_names)]['name'].unique()))
print(len(df_fbref[df_fbref['player'].isin(common_names)]['player'].unique()))

8110
8110


In [86]:
merged_df = pd.merge(df, df_fbref, on=['player', 'season'])

In [92]:
merged_df

,player,age_x,team_x,height(cm),weight(kg),season,overall_rating,potential,foot,best_overall,growth,total_attacking,crossing,finishing,heading_accuracy,short_passing,volleys,total_skill,dribbling,curve,fk_accuracy,long_passing,ball_control,total_movement,acceleration,sprint_speed,agility,reactions,balance,total_power,shot_power,jumping,stamina,strength,long_shots,...,Performance_PKwon,Performance_PKcon,Performance_OG,Expected_xG,Expected_npxG,Expected_xAG,Expected_npxG+xAG,Progression_PrgC,Progression_PrgP,Progression_PrgR,Per 90 Minutes_xG,Per 90 Minutes_xAG,Per 90 Minutes_xG+xAG,Per 90 Minutes_npxG,Per 90 Minutes_npxG+xAG,Standard_FK,Expected_npxG/Sh,Expected_G-xG,Expected_np:G-xG,xAG,Expected_xA,Expected_A-xAG,PrgP,Carries_PrgC,Receiving_Rec,Receiving_PrgR,Team Success (xG)_onxG,Team Success (xG)_onxGA,Team Success (xG)_xG+/-,Team Success (xG)_xG+/-90,Team Success (xG)_On-Off,Performance_Recov,Aerial Duels_Won,Aerial Duels_Lost,Aerial Duels_Won%
0,e. haaland,22,City,195,94,2024,91,94,1,93,3,393,47,96,83,77,90,363,79,77,62,63,82,420,82,94,78,94,72,442,94,93,76,93,86,...,0.0,0.0,0.0,22.0,18.8,3.0,21.8,24.0,20.0,124.0,0.72,0.10,0.82,0.62,0.72,1.0,0.19,0.0,0.2,3.0,2.0,0.0,20.0,24.0,478.0,124.0,56.2,40.5,15.7,0.52,-0.10,34.0,57.0,50.0,53.3
1,k. mbappé,24,Germain,182,75,2024,91,94,2,93,3,415,78,94,73,86,84,405,93,80,69,71,92,462,97,97,93,93,82,426,90,88,88,77,83,...,3.0,0.0,0.0,25.9,18.6,7.7,26.3,150.0,140.0,386.0,0.80,0.24,1.04,0.58,0.81,3.0,0.12,5.1,5.4,7.7,6.6,-4.7,140.0,150.0,1355.0,386.0,69.0,35.2,33.8,1.05,1.27,36.0,4.0,6.0,40.0
2,rodri,27,City,191,82,2024,90,92,2,90,2,386,71,74,81,92,68,410,84,83,63,90,90,356,63,65,68,93,67,435,92,83,89,83,88,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,0.00,0.06,0.06,0.00,0.06,0.0,NaN,0.0,0.0,0.0,0.1,0.0,4.0,1.0,56.0,1.0,2.2,0.3,1.9,2.36,1.86,7.0,1.0,0.0,100.0
3,h. kane,29,nchen,188,85,2024,90,90,2,90,0,440,80,95,89,87,89,404,82,82,65,88,87,369,67,70,66,93,73,433,93,87,81,85,87,...,2.0,0.0,0.0,20.3,13.3,5.3,18.6,29.0,97.0,143.0,0.77,0.20,0.97,0.50,0.70,6.0,0.13,5.7,3.7,5.3,5.6,3.7,97.0,29.0,716.0,143.0,65.0,17.8,47.2,1.79,0.60,61.0,29.0,20.0,59.2
4,r. lewandowski,34,Barcelona,185,81,2024,90,90,2,90,0,433,76,93,91,84,89,410,86,79,85,70,90,404,76,75,77,93,83,432,91,92,76,89,84,...,0.0,0.0,0.0,27.1,24.0,2.3,26.3,27.0,48.0,164.0,0.92,0.08,0.99,0.81,0.89,5.0,0.22,-0.1,0.0,2.3,1.5,-0.3,48.0,27.0,667.0,164.0,63.4,31.2,32.2,1.09,-0.98,45.0,37.0,39.0,48.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23446,c. capuano,24,FC,185,74,2007,51,54,1,51,3,185,44,32,59,50,0,193,30,0,48,63,52,164,57,64,0,43,0,215,40,0,60,42,73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23447,p. ebert,19,BSC,175,72,2007,49,66,1,49,17,183,48,34,52,49,0,154,47,0,23,33,51,151,54,59,0,38,0,166,47,0,40,41,38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23448,g. obertan,17,Bordeaux,186,79,2007,48,85,2,48,37,165,32,52,49,32,0,129,48,0,19,32,30,164,67,66,0,31,0,165,53,0,28,35,49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23449,m. izco,23,Catania,170,68,2007,48,62,2,48,14,180,46,42,50,42,0,195,51,0,53,46,45,162,52,47,0,63,0,214,52,0,55,48,59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
